<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_21_Keypad_Conundrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
input = """029A
980A
179A
456A
379A"""

In [111]:
input = """539A
964A
803A
149A
789A"""

In [113]:
from collections import deque

def get_direction(from_pos, to_pos):
    """Get the direction command to move from one position to another"""
    from_row, from_col = from_pos
    to_row, to_col = to_pos

    if from_row > to_row:
        return '^' * (from_row - to_row)  # Multiple steps up
    if from_row < to_row:
        return 'v' * (to_row - from_row)  # Multiple steps down
    if from_col > to_col:
        return '<' * (from_col - to_col)  # Multiple steps left
    if from_col < to_col:
        return '>' * (to_col - from_col)  # Multiple steps right
    return ''

def find_path(sequence, keypad):
    """Process a sequence of buttons and return possible paths of commands"""
    gap_pos = keypad['X']
    current_pos = keypad['A']
    all_paths = ['']  # Start with empty path

    for char in sequence.strip():
        if char not in keypad or char == 'X':
            continue

        target_pos = keypad[char]
        curr_row, curr_col = current_pos
        target_row, target_col = target_pos

        # Check if each path crosses the gap
        vert_crosses_gap = (curr_col == gap_pos[1] and
                           min(curr_row, target_row) <= gap_pos[0] <= max(curr_row, target_row))
        horz_crosses_gap = (curr_row == gap_pos[0] and
                           min(curr_col, target_col) <= gap_pos[1] <= max(curr_col, target_col))

        new_paths = []
        for base_path in all_paths:
            # Add vertical-first path if valid
            if not vert_crosses_gap:
                vert_path = base_path
                if curr_row != target_row:
                    vert_path += get_direction(current_pos, (target_row, curr_col))
                if curr_col != target_col:
                    vert_path += get_direction((target_row, curr_col), target_pos)
                vert_path += 'A'
                new_paths.append(vert_path)

            # Add horizontal-first path if valid and different
            if not horz_crosses_gap:
                horz_path = base_path
                if curr_col != target_col:
                    horz_path += get_direction(current_pos, (curr_row, target_col))
                if curr_row != target_row:
                    horz_path += get_direction((curr_row, target_col), target_pos)
                horz_path += 'A'
                # Only add if it's different from vertical path
                if horz_path not in new_paths:
                    new_paths.append(horz_path)

        all_paths = new_paths
        current_pos = target_pos

    return all_paths

def reverse_path(path, keypad):
    """
    Given a path string (e.g. "^^<<A>A") and a keypad layout,
    returns the sequence of buttons that were pressed
    """
    current_pos = keypad['A']  # Start at 'A' button
    buttons = []

    i = 0
    while i < len(path):
        moves = []
        while i < len(path) and path[i] in '^v<>':
            moves.append(path[i])
            i += 1

        for move in moves:
            row, col = current_pos
            if move == '^':
                current_pos = (row - 1, col)
            elif move == 'v':
                current_pos = (row + 1, col)
            elif move == '<':
                current_pos = (row, col - 1)
            elif move == '>':
                current_pos = (row, col + 1)

        if i < len(path) and path[i] == 'A':
            for button, pos in keypad.items():
                if pos == current_pos and button != 'X':
                    buttons.append(button)
                    break
            i += 1

    return ''.join(buttons)

num_keypad = {
    '7': (0, 0), '8': (0, 1), '9': (0, 2),
    '4': (1, 0), '5': (1, 1), '6': (1, 2),
    '1': (2, 0), '2': (2, 1), '3': (2, 2),
    'X': (3, 0), '0': (3, 1), 'A': (3, 2)
}
dir_keypad = {
    'X': (0, 0), '^': (0, 1), 'A': (0, 2),
    '<': (1, 0), 'v': (1, 1), '>': (1, 2),
}
total = 0
for x in input.split():
  n = int(x[:-1])
  # print(x)
  x = find_path(x, num_keypad)
  # print(x)
  x = [s for si in x for s in find_path(si, dir_keypad)]
  # print(x)
  x = [s for si in x for s in find_path(si, dir_keypad)]
  # print(x)
  print(min(len(s) for s in x))
  total += min(len(s) for s in x) * n
total

70
72
76
76
66


231564